# Guitar Amplifier Emulation Research

## Setup

In [ ]:
import numpy as np
import seaborn as sns
import keras.backend as K
import matplotlib.pyplot as plt
import librosa
import tensorflow as tf

from keras.optimizers import Adam
from keras.models import Model
from keras.layers import LSTM, Dense, Input, Add, Conv1D, BatchNormalization, Dropout
from keras.callbacks import BackupAndRestore, CSVLogger, TensorBoard, ModelCheckpoint

from callbacks import CheckpointAndSave, CosineAnnealingScheduler
from metrics import error_to_signal
from dataset import DirectSTFTLoader, DirectWaveformLoader, WindowWaveformLoader, DataSplitter, WindowSTFTLoader
from utils import save_attempt_data

%load_ext autoreload
%autoreload 2

Verify GPU support

In [ ]:
tf.config.list_physical_devices()

Defining constants

In [ ]:
INPUT_PATH = "data/float32/clean.wav"
OUTPUT_PATH = "data/float32/amp.wav"

TEST_INPUT_PATH = "data/float32/test_in.wav"
TEST_OUTPUT_PATH = "data/float32/test_out.wav"

WORK_DIR = "models"

## Baseline

Basic LSTM, $y_k = f(c_{k - 1}, h_{k - 1}, x_k), k = [0, n)$

Defining required values

In [ ]:
batch_size = 2 ** 5 # in samples
inference_batch_size = 1 # in samples
input_size = 22050 # in samples
duration = 15 * 60 # in seconds
epochs = 750

Creating dataset from audio

In [ ]:
train, valid = DataSplitter(input_path=INPUT_PATH,
                            output_path=OUTPUT_PATH,
                            duration=duration,
                            batch_size=batch_size,
                            input_size=input_size,
                            preemphasis=True).split(DirectWaveformLoader)

demo = DirectWaveformLoader(input_path=INPUT_PATH,
                            output_path=OUTPUT_PATH,
                            batch_size=inference_batch_size,
                            input_size=input_size, 
                            duration=10,
                            offset=86,
                            preemphasis=False)

In [ ]:
learning_rate = 0.001
hidden_units = 36

In [ ]:
K.clear_session()

input_layer = Input(batch_shape=(batch_size, input_size, 1))
rnn = LSTM(hidden_units, return_sequences=True)(input_layer)
rnn = Dense(1)(rnn)
add = Add()([input_layer, rnn])

model = Model(inputs=input_layer, outputs=[rnn])

model.compile(optimizer=Adam(learning_rate), loss='mse', metrics=['mse', error_to_signal, 'mae'])

model.summary()

In [ ]:
model_name = input()

if model_name == "":
  raise ValueError("Enter model name")

In [ ]:
subfolder = f"{WORK_DIR}/{model_name}"
save_attempt_data(subfolder, model)

In [ ]:
callbacks = [
  BackupAndRestore(f'{subfolder}/backup'),
  CheckpointAndSave(f'{subfolder}/mse_checkpoint/', demo, model, inference_batch_size, save_best_only=True, save_weights_only=True, monitor='val_mse', verbose=1),
  CheckpointAndSave(f'{subfolder}/ets_checkpoint/', demo, model, inference_batch_size, save_best_only=True, save_weights_only=True, monitor='val_error_to_signal', verbose=1),
  CheckpointAndSave(f'{subfolder}/mae_checkpoint/', demo, model, inference_batch_size, save_best_only=True, save_weights_only=True, monitor='val_mae', verbose=1),
  TensorBoard(log_dir=f'{subfolder}/logs'),
  CSVLogger(f'{subfolder}/logs.csv', append=True),
]

In [ ]:
model.fit(train, validation_data=valid, shuffle=True, epochs=epochs, callbacks=callbacks)

## Baseline 2

Defining required values

In [ ]:
batch_size = 2 ** 14 # in samples
input_size = 150 # in samples
epochs = 100
duration = 30 * 60 # in seconds

Creating dataset from audio

$a_n = f(a_{n - 1}, a_{n - 2}, \dots, a_{n - k})$

In [ ]:
train, valid = DataSplitter(input_path=INPUT_PATH, 
                            output_path=OUTPUT_PATH,
                            batch_size=batch_size,
                            duration=duration,
                            input_size=input_size,
                            preemphasis=True).split(WindowWaveformLoader)

demo = WindowWaveformLoader(input_path=TEST_INPUT_PATH,
                            output_path=TEST_OUTPUT_PATH,
                            batch_size=batch_size,
                            input_size=input_size,
                            duration=10,
                            offset=86,
                            preemphasis=False)

In [ ]:
learning_rate = 0.004
conv1d_strides = 3
conv1d_filters = 32
hidden_units = 36
kernel_size = 5

K.clear_session()

input_layer = Input((input_size, 1))
conv1d = Conv1D(conv1d_filters, kernel_size, strides=conv1d_strides, activation=None, padding='same')(input_layer)
conv1d = Conv1D(conv1d_filters, kernel_size, strides=conv1d_strides, activation=None, padding='same')(conv1d)
rnn = LSTM(hidden_units)(conv1d)
rnn = Dropout(0.2)(rnn)
rnn = BatchNormalization()(rnn)
rnn = Dense(1, activation=None)(rnn)

model = Model(inputs=input_layer, outputs=[rnn])

model.compile(optimizer=Adam(learning_rate), loss='mse', metrics=['mse', error_to_signal, 'mae'])

model.summary()

In [ ]:
model_name = input()

if model_name == "":
  raise ValueError("Enter model name")

subfolder = f"{WORK_DIR}/{model_name}"
save_attempt_data(subfolder, model)

In [ ]:
callbacks = [
  CosineAnnealingScheduler(5, 4e-3, 1e-3),
  BackupAndRestore(f'{subfolder}/backup'),
  CheckpointAndSave(f'{subfolder}/mse_checkpoint/', demo, model, batch_size, save_best_only=True, save_weights_only=True, monitor='val_mse', verbose=1),
  CheckpointAndSave(f'{subfolder}/ets_checkpoint/', demo, model, batch_size, save_best_only=True, save_weights_only=True, monitor='val_error_to_signal', verbose=1),
  CheckpointAndSave(f'{subfolder}/mae_checkpoint/', demo, model, batch_size, save_best_only=True, save_weights_only=True, monitor='val_mae', verbose=1),
  TensorBoard(log_dir=f'{subfolder}/logs'),
  CSVLogger(f'{subfolder}/logs.csv', append=True),
]

In [ ]:
model.fit(train, validation_data=valid, shuffle=True, epochs=epochs, callbacks=callbacks)

In [ ]:
pred = model.predict(demo)

## STFT Transform approach

Defining required values

In [ ]:
batch_size = 2 ** 5
inference_batch_size = 2 ** 5
input_size = 4410
duration = 3 * 60 # in seconds
epochs = 100

n_fft = 512
hop_length = 128

Creating dataset from audio

In [ ]:
train, valid = DataSplitter(input_path=INPUT_PATH, 
                            output_path=OUTPUT_PATH,
                            batch_size=batch_size,
                            duration=duration,
                            input_size=input_size,
                            preemphasis=True).split(DirectSTFTLoader, n_fft=512, hop_length=128)

demo = DirectSTFTLoader(input_path=TEST_INPUT_PATH, 
                       output_path=TEST_OUTPUT_PATH, 
                       batch_size=inference_batch_size,
                       input_size=input_size, 
                       duration=10, 
                       offset=86,
                       preemphasis=False)

In [ ]:
train[0][0].shape

In [ ]:
feature_size = train[0][0].shape[2]
learning_rate = 0.001
hidden_units = 64

K.clear_session()

input_layer = Input(batch_shape=train[0][0].shape)
rnn = LSTM(hidden_units, return_sequences=True)(input_layer)
rnn = Dense(train[0][0].shape[2])(rnn)
add = Add()([input_layer, rnn])

model = Model(inputs=input_layer, outputs=[rnn])

model.compile(optimizer=Adam(learning_rate), loss='mse', metrics=['mse', 'mae'])

model.summary()

In [ ]:
model_name = input()

if model_name == "":
  raise ValueError("Enter model name")

subfolder = f"{WORK_DIR}/{model_name}"
save_attempt_data(subfolder, model)

In [ ]:
callbacks = [
  CosineAnnealingScheduler(5, 4e-3, 1e-3),
  BackupAndRestore(f'{subfolder}/backup'),
  ModelCheckpoint(f'{subfolder}/mse_checkpoint/', save_best_only=True, save_weights_only=True, monitor='val_mse', verbose=1),
  ModelCheckpoint(f'{subfolder}/mae_checkpoint/', save_best_only=True, save_weights_only=True, monitor='val_mae', verbose=1),
  TensorBoard(log_dir=f'{subfolder}/logs'),
  CSVLogger(f'{subfolder}/logs.csv', append=True),
]

In [ ]:
model.fit(train, validation_data=valid, shuffle=True, epochs=epochs, callbacks=callbacks)

In [ ]:
pred = model.predict(train, batch_size=batch_size, verbose=1)

In [ ]:
from utils import output_to_audio

result = output_to_audio(pred, n_fft=n_fft, hop_length=hop_length)

In [ ]:
from scipy.io import wavfile
wavfile.write("out.wav", 44100, result.flatten())

## Baseline 2

Defining required values

In [ ]:
batch_size = 2 ** 12
input_size = 441
epochs = 100
duration = 3 * 60 # in seconds

Creating dataset from audio

$a_n = f(a_{n - 1}, a_{n - 2}, \dots, a_{n - k})$

In [ ]:
train, valid = DataSplitter(input_path=INPUT_PATH,
                            output_path=OUTPUT_PATH,
                            batch_size=batch_size,
                            duration=duration,
                            input_size=input_size,
                            preemphasis=True).split(WindowSTFTLoader, n_fft=128, hop_length=32)

demo = WindowSTFTLoader(input_path=TEST_INPUT_PATH,
                        output_path=TEST_OUTPUT_PATH,
                        batch_size=batch_size,
                        input_size=input_size,
                        duration=10,
                        offset=86,
                        preemphasis=False)

In [ ]:
first_batch = train[0][0]

In [ ]:
first_batch.shape

In [ ]:
learning_rate = 0.004
conv1d_strides = 6
conv1d_filters = 32
hidden_units = 36
kernel_size = 12

K.clear_session()

input_layer = Input((first_batch.shape[1], first_batch.shape[2]))
rnn = LSTM(hidden_units)(input_layer)
rnn = Dropout(0.2)(rnn)
rnn = BatchNormalization()(rnn)
rnn = Dense(1, activation=None)(rnn)

model = Model(inputs=input_layer, outputs=[rnn])

model.compile(optimizer=Adam(learning_rate), loss='mse', metrics=['mse', error_to_signal, 'mae'])

model.summary()

In [ ]:
model_name = input()

if model_name == "":
  raise ValueError("Enter model name")

subfolder = f"{WORK_DIR}/{model_name}"
save_attempt_data(subfolder, model)

In [ ]:
callbacks = [
  CosineAnnealingScheduler(5, 4e-3, 1e-3),
  BackupAndRestore(f'{subfolder}/backup'),
  # CheckpointAndSave(f'{subfolder}/mse_checkpoint/', demo, model, batch_size, save_best_only=True, save_weights_only=True, monitor='val_mse', verbose=1),
  # CheckpointAndSave(f'{subfolder}/ets_checkpoint/', demo, model, batch_size, save_best_only=True, save_weights_only=True, monitor='val_error_to_signal', verbose=1),
  # CheckpointAndSave(f'{subfolder}/mae_checkpoint/', demo, model, batch_size, save_best_only=True, save_weights_only=True, monitor='val_mae', verbose=1),
  TensorBoard(log_dir=f'{subfolder}/logs'),
  CSVLogger(f'{subfolder}/logs.csv', append=True),
]

In [ ]:
model.fit(train, validation_data=valid, shuffle=True, epochs=epochs, callbacks=callbacks)

In [ ]:
pred = model.predict(demo)